In [1]:
import pandas as pd
import numpy as np 

In [2]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,41 mins 11 secs
H2O cluster version:,3.10.0.9
H2O cluster version age:,6 months and 14 days !!!
H2O cluster name:,H2O_from_python_acorn_0346ok
H2O cluster total nodes:,1
H2O cluster free memory:,1.555 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"
H2O connection url:,http://localhost:54321
H2O connection proxy:,None


파일불러오기

In [3]:
data= h2o.import_file("pump_train_set.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
finaltest = h2o.import_file("pump_test_set.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [7]:
submitFormat = h2o.import_file("SubmissionFormat.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [8]:
submit_id = submitFormat['id']

In [9]:
data.shape

(59400, 21)

In [10]:
data.head()

amount_tsh,funder,gps_height,installer,longitude,latitude,basin,region,population,public_meeting,scheme_management,permit,construction_year,extraction_type,management,payment,water_quality,quantity,source,waterpoint_type,status_group
8.69968,Roman,1390,others,34.9381,-9.85632,Lake Nyasa,Iringa,4.70048,True,VWC,False,1999,gravity,vwc,pay annually,soft,enough,spring,communal standpipe,functional
0,others,1399,others,34.6988,-2.14747,Lake Victoria,Mara,5.63835,unknown,Other,True,2010,gravity,wug,never pay,soft,insufficient,rainwater harvesting,communal standpipe,functional
3.2581,others,686,World vision,37.4607,-3.82133,Pangani,Manyara,5.52545,True,VWC,True,2009,gravity,vwc,pay per bucket,soft,enough,dam,communal standpipe multiple,functional
0,Unicef,263,UNICEF,38.4862,-11.1553,Ruvuma / Southern Coast,Mtwara,4.07754,True,VWC,True,1986,submersible,vwc,never pay,soft,dry,machine dbh,communal standpipe multiple,non functional
0,others,0,Artisan,31.1308,-1.82536,Lake Victoria,Kagera,0,True,unknown,True,1990,gravity,other,never pay,soft,seasonal,rainwater harvesting,communal standpipe,functional
3.04452,Mkinga Distric Coun,0,DWE,39.1728,-4.76559,Pangani,Tanga,0.693147,True,VWC,True,2009,submersible,vwc,pay per bucket,salty,enough,other,communal standpipe multiple,functional
0,Dwsp,0,DWSP,33.3624,-3.76636,Internal,Shinyanga,0,True,VWC,True,1990,swn 80,vwc,never pay,soft,enough,machine dbh,hand pump,non functional
0,Rwssp,0,DWE,32.6206,-4.2262,Lake Tanganyika,Shinyanga,0,True,unknown,True,1990,nira/tanira,wug,unknown,milky,enough,shallow well,hand pump,non functional
0,Wateraid,0,others,32.7111,-5.14671,Lake Tanganyika,Tabora,0,True,VWC,True,1990,india mark ii,vwc,never pay,salty,seasonal,machine dbh,hand pump,non functional
0,others,0,Artisan,30.627,-1.25705,Lake Victoria,Kagera,0,True,unknown,True,1990,nira/tanira,vwc,never pay,soft,enough,shallow well,hand pump,functional


In [11]:
data['status_group'] = data['status_group'].asfactor()  #encode the binary repsonse as a factor
data['status_group'].levels() 

[['functional', 'functional needs repair', 'non functional']]

In [12]:
splits = data.split_frame(ratios=[0.7, 0.15], seed=1)  

train = splits[0]
valid = splits[1]
test = splits[2]

In [13]:

print(train.nrow)
print(valid.nrow)
print (test.nrow)

41601
8827
8972


In [14]:
y = 'status_group'
x = list(data.columns)

In [15]:
x.remove(y)

In [16]:
x

['amount_tsh',
 'funder',
 'gps_height',
 'installer',
 'longitude',
 'latitude',
 'basin',
 'region',
 'population',
 'public_meeting',
 'scheme_management',
 'permit',
 'construction_year',
 'extraction_type',
 'management',
 'payment',
 'water_quality',
 'quantity',
 'source',
 'waterpoint_type']

In [46]:
from h2o.estimators.random_forest import H2ORandomForestEstimator

In [47]:
rf_fit1 = H2ORandomForestEstimator(model_id='rf_fit1', ntrees=100, seed=1)
rf_fit1.train(x=x, y=y, training_frame=train)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [48]:
rf_fit2 = H2ORandomForestEstimator(model_id='rf_fit2', ntrees=200, seed=1)
rf_fit2.train(x=x, y=y, training_frame=train)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [49]:
rf_fit3 = H2ORandomForestEstimator(model_id='rf_fit3', ntrees=300, seed=1)
rf_fit3.train(x=x, y=y, training_frame=train)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [50]:
rf_fit4 = H2ORandomForestEstimator(model_id='rf_fit4', ntrees=400, seed=1)
rf_fit4.train(x=x, y=y, training_frame=train)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [51]:
rf_perf1 = rf_fit1.model_performance(test)
rf_perf2 = rf_fit2.model_performance(test)
rf_perf3 = rf_fit3.model_performance(test)
rf_perf4 = rf_fit4.model_performance(test)

In [62]:
rf_perf1 #0.7993


ModelMetricsMultinomial: drf
** Reported on test data. **

MSE: 0.15867419689792517
RMSE: 0.3983392987114442
LogLoss: 0.5000207789922031
Mean Per-Class Error: 0.3567414046824527
Confusion Matrix: vertical: actual; across: predicted



functional,functional needs repair,non functional,Error,Rate
4396.0,73.0,317.0,0.0814877,"390 / 4,786"
362.0,176.0,101.0,0.7245696,463 / 639
913.0,24.0,2610.0,0.2641669,"937 / 3,547"
5671.0,273.0,3028.0,0.1995096,"1,790 / 8,972"


Top-3 Hit Ratios: 


k,hit_ratio
1,0.8004904
2,0.9569773
3,1.0


In [63]:
rf_perf2


ModelMetricsMultinomial: drf
** Reported on test data. **

MSE: 0.15853106846420098
RMSE: 0.3981596017480942
LogLoss: 0.49490702851516166
Mean Per-Class Error: 0.3606018862949458
Confusion Matrix: vertical: actual; across: predicted



functional,functional needs repair,non functional,Error,Rate
4393.0,75.0,318.0,0.0821145,"393 / 4,786"
371.0,169.0,99.0,0.7355243,470 / 639
916.0,21.0,2610.0,0.2641669,"937 / 3,547"
5680.0,265.0,3027.0,0.2006242,"1,800 / 8,972"


Top-3 Hit Ratios: 


k,hit_ratio
1,0.7993758
2,0.9584262
3,1.0


In [64]:
rf_perf3 


ModelMetricsMultinomial: drf
** Reported on test data. **

MSE: 0.15842838516065175
RMSE: 0.39803063344502987
LogLoss: 0.4936193899673478
Mean Per-Class Error: 0.3604528043489518
Confusion Matrix: vertical: actual; across: predicted



functional,functional needs repair,non functional,Error,Rate
4389.0,74.0,323.0,0.0829503,"397 / 4,786"
370.0,170.0,99.0,0.7339593,469 / 639
916.0,22.0,2609.0,0.2644488,"938 / 3,547"
5675.0,266.0,3031.0,0.2010700,"1,804 / 8,972"


Top-3 Hit Ratios: 


k,hit_ratio
1,0.79893
2,0.9587606
3,1.0


In [84]:
rf_perf4


ModelMetricsMultinomial: drf
** Reported on test data. **

MSE: 0.15825887208886605
RMSE: 0.39781763672424836
LogLoss: 0.49279767518350076
Mean Per-Class Error: 0.36065054338861396
Confusion Matrix: vertical: actual; across: predicted



functional,functional needs repair,non functional,Error,Rate
4395.0,76.0,315.0,0.0816966,"391 / 4,786"
372.0,169.0,98.0,0.7355243,470 / 639
917.0,22.0,2608.0,0.2647308,"939 / 3,547"
5684.0,267.0,3021.0,0.2006242,"1,800 / 8,972"


Top-3 Hit Ratios: 


k,hit_ratio
1,0.7993758
2,0.9592064
3,1.0


In [94]:
rf_fit5 = H2ORandomForestEstimator(model_id='rf_fit5', seed=1, nfolds=5)
rf_fit5.train(x=x, y=y, training_frame=data)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [90]:
rf_fit5_1 = H2ORandomForestEstimator(model_id='rf_fit5_1', seed=1, nfolds=5,ntrees=100)
rf_fit5_1.train(x=x, y=y, training_frame=data)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [52]:
rf_fit5_2 = H2ORandomForestEstimator(model_id='rf_fit5_2', seed=1, nfolds=5,ntrees=200)
rf_fit5_2.train(x=x, y=y, training_frame=data)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [95]:
rf_perf5 = rf_fit5.model_performance(test)

In [96]:
rf_perf5_1=rf_fit5_1.model_performance(test)

In [102]:
rf_perf5_2=rf_fit5_2.model_performance(test)

In [54]:
rf_fit5_2.varimp

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  rf_fit5_2
Model Summary: 


,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,200.0,600.0,24462671.0000000,20.0,20.0,20.0,1594.0,4357.0,3067.63




ModelMetricsMultinomial: drf
** Reported on train data. **

MSE: 0.15447768301612494
RMSE: 0.39303649069281715
LogLoss: 0.48443244171905936
Mean Per-Class Error: 0.3529716821395847
Confusion Matrix: vertical: actual; across: predicted



functional,functional needs repair,non functional,Error,Rate
29755.0,407.0,2097.0,0.0776217,"2,504 / 32,259"
2612.0,1188.0,517.0,0.7248089,"3,129 / 4,317"
5654.0,200.0,16970.0,0.2564844,"5,854 / 22,824"
38021.0,1795.0,19584.0,0.1933838,"11,487 / 59,400"


Top-3 Hit Ratios: 


...

Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error
,2017-05-17 14:47:59,3 min 33.926 sec,0.0,nan,nan,nan
,2017-05-17 14:48:00,3 min 34.101 sec,1.0,0.5421211,7.8329258,0.2899694
,2017-05-17 14:48:00,3 min 34.289 sec,2.0,0.5373064,7.5045630,0.2894803
,2017-05-17 14:48:00,3 min 34.473 sec,3.0,0.5087717,6.0955456,0.2709099
,2017-05-17 14:48:00,3 min 34.644 sec,4.0,0.4965550,5.3444079,0.2648228
---,---,---,---,---,---,---
,2017-05-17 14:48:20,3 min 54.203 sec,111.0,0.3935262,0.4935152,0.1939731
,2017-05-17 14:48:24,3 min 58.308 sec,134.0,0.3933529,0.4900055,0.1948485
,2017-05-17 14:48:28,4 min 2.359 sec,157.0,0.3932805,0.4876577,0.1946465
,2017-05-17 14:48:32,4 min 6.401 sec,180.0,0.3931066,0.4851648,0.1938384



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
quantity,562840.5,1.0,0.1848549
funder,309902.5625000,0.5506046,0.1017820
extraction_type,299910.4375000,0.5328516,0.0985002
installer,253254.3593750,0.4499576,0.0831769
waterpoint_type,245694.2031250,0.4365255,0.0806939
region,154929.1718750,0.2752630,0.0508837
payment,150111.9062500,0.2667042,0.0493016
construction_year,138624.8125000,0.2462950,0.0455288
latitude,113977.8281250,0.2025047,0.0374340
source,112570.2421875,0.2000038,0.0369717


<bound method ModelBase.varimp of >

In [66]:
rf_perf5 # 최종결과 0.8011


ModelMetricsMultinomial: drf
** Reported on test data. **

MSE: 0.10096282407181895
RMSE: 0.31774647767020003
LogLoss: 0.3254260643276188
Mean Per-Class Error: 0.24035689411476455
Confusion Matrix: vertical: actual; across: predicted



functional,functional needs repair,non functional,Error,Rate
4634.0,24.0,128.0,0.0317593,"152 / 4,786"
279.0,305.0,55.0,0.5226917,334 / 639
579.0,12.0,2956.0,0.1666197,"591 / 3,547"
5492.0,341.0,3139.0,0.1200401,"1,077 / 8,972"


Top-3 Hit Ratios: 


k,hit_ratio
1,0.8799599
2,0.9862907
3,1.0


In [97]:
rf_perf5_1 #최종 0.8028


ModelMetricsMultinomial: drf
** Reported on test data. **

MSE: 0.10078811497308289
RMSE: 0.317471439617933
LogLoss: 0.3254269578886835
Mean Per-Class Error: 0.24214701759316717
Confusion Matrix: vertical: actual; across: predicted



functional,functional needs repair,non functional,Error,Rate
4639.0,25.0,122.0,0.0307146,"147 / 4,786"
285.0,300.0,54.0,0.5305164,339 / 639
575.0,11.0,2961.0,0.1652100,"586 / 3,547"
5499.0,336.0,3137.0,0.1194828,"1,072 / 8,972"


Top-3 Hit Ratios: 


k,hit_ratio
1,0.8805172
2,0.9860678
3,1.0


In [103]:
rf_perf5_2 # 최종0.8026


ModelMetricsMultinomial: drf
** Reported on test data. **

MSE: 0.10125625533774274
RMSE: 0.3182078806971046
LogLoss: 0.32719490423016934
Mean Per-Class Error: 0.2394514755327893
Confusion Matrix: vertical: actual; across: predicted



functional,functional needs repair,non functional,Error,Rate
4647.0,23.0,116.0,0.0290430,"139 / 4,786"
285.0,305.0,49.0,0.5226917,334 / 639
580.0,11.0,2956.0,0.1666197,"591 / 3,547"
5512.0,339.0,3121.0,0.1185912,"1,064 / 8,972"


Top-3 Hit Ratios: 


k,hit_ratio
1,0.8814088
2,0.9867365
3,0.9999999


# Prediction

In [129]:
prediction=dl_fit1.predict(test_data=finaltest)

deeplearning prediction progress: |███████████████████████████████████████| 100%


In [130]:
prediction[:1].shape

(14850, 1)

# how to submit

In [131]:
submission = pd.concat((h2o.as_list(submit_id), h2o.as_list(prediction[:1])),axis=1, ignore_index=True)

submission.columns = ['id', 'status_group']

submission = submission.set_index(['id', 'status_group'])

submission.to_csv('h2o_py_dlfit1.csv')

In [19]:
    from h2o.estimators.gbm import H2OGradientBoostingEstimator

In [26]:
gbm_fit1 = H2OGradientBoostingEstimator(model_id='gbm_fit1', seed=1)
gbm_fit1.train(x=x, y=y, training_frame=train)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [20]:
gbm_fit2 = H2OGradientBoostingEstimator(model_id='gbm_fit2',ntrees=200,seed=1)
gbm_fit2.train(x=x, y=y, training_frame=train)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [24]:
gbm_fit5 = H2OGradientBoostingEstimator(model_id='gbm_fit5', ntrees=500, seed=1)
gbm_fit5.train(x=x, y=y, training_frame=train)

gbm Model Build progress: |███████████████████████████████████████████████| 100%




gbm_fit5_1 = H2OGradientBoostingEstimator(model_id='gbm_fit3', 
                                        ntrees=500, 
                                        score_tree_interval=5,     #used for early stopping
                                        stopping_rounds=3,         #used for early stopping
                                         #used for early stopping
                                        stopping_tolerance=0.0005, #used for early stopping
                                        seed=1)


gbm_fit3.train(x=x, y=y, training_frame=train, validation_frame=valid)

In [27]:
gbm_perf1 = gbm_fit1.model_performance(test)
gbm_perf2 = gbm_fit2.model_performance(test)
gbm_perf5 = gbm_fit5.model_performance(test)


In [28]:
print(gbm_perf1)



ModelMetricsMultinomial: gbm
** Reported on test data. **

MSE: 0.18647213808306104
RMSE: 0.43182419812125056
LogLoss: 0.5616226423060752
Mean Per-Class Error: 0.4096996833871021
Confusion Matrix: vertical: actual; across: predicted



functional,functional needs repair,non functional,Error,Rate
4382.0,49.0,355.0,0.0844129,"404 / 4,786"
425.0,114.0,100.0,0.8215962,525 / 639
1125.0,21.0,2401.0,0.3230899,"1,146 / 3,547"
5932.0,184.0,2856.0,0.2312751,"2,075 / 8,972"


Top-3 Hit Ratios: 


k,hit_ratio
1,0.7687249
2,0.9506242
3,1.0


In [29]:
gbm_perf2 #최종 0.7911


ModelMetricsMultinomial: gbm
** Reported on test data. **

MSE: 0.16863521283825125
RMSE: 0.41065217987763225
LogLoss: 0.5175079594290852
Mean Per-Class Error: 0.3736756466799503
Confusion Matrix: vertical: actual; across: predicted



functional,functional needs repair,non functional,Error,Rate
4349.0,81.0,356.0,0.0913080,"437 / 4,786"
379.0,159.0,101.0,0.7511737,480 / 639
952.0,36.0,2559.0,0.2785452,"988 / 3,547"
5680.0,276.0,3016.0,0.2123272,"1,905 / 8,972"


Top-3 Hit Ratios: 


k,hit_ratio
1,0.7876728
2,0.9557512
3,1.0


In [115]:
gbm_fit2.scoring_history()

,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error
0,,2017-05-17 11:55:07,0.001 sec,0.0,0.666667,1.098612,0.553761
1,,2017-05-17 11:55:07,0.080 sec,1.0,0.638078,1.017633,0.271893
2,,2017-05-17 11:55:07,0.171 sec,2.0,0.613168,0.953530,0.264873
3,,2017-05-17 11:55:07,0.252 sec,3.0,0.591262,0.900955,0.261244
4,,2017-05-17 11:55:07,0.316 sec,4.0,0.572305,0.857728,0.261508
5,,2017-05-17 11:55:07,0.382 sec,5.0,0.555570,0.820993,0.260835
6,,2017-05-17 11:55:07,0.443 sec,6.0,0.541037,0.789735,0.258335
7,,2017-05-17 11:55:07,0.504 sec,7.0,0.528240,0.762764,0.257037
8,,2017-05-17 11:55:07,0.565 sec,8.0,0.517563,0.740524,0.256076
9,,2017-05-17 11:55:07,0.625 sec,9.0,0.508025,0.720754,0.252854


In [30]:
gbm_perf5  #최종 0.7911


ModelMetricsMultinomial: gbm
** Reported on test data. **

MSE: 0.16303793087277552
RMSE: 0.40377955727448056
LogLoss: 0.506564881457785
Mean Per-Class Error: 0.35773622555233936
Confusion Matrix: vertical: actual; across: predicted



functional,functional needs repair,non functional,Error,Rate
4305.0,91.0,390.0,0.1005015,"481 / 4,786"
362.0,183.0,94.0,0.7136150,456 / 639
879.0,40.0,2628.0,0.2590922,"919 / 3,547"
5546.0,314.0,3112.0,0.2068658,"1,856 / 8,972"


Top-3 Hit Ratios: 


k,hit_ratio
1,0.7931342
2,0.957646
3,1.0


In [32]:
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

In [35]:
dl_fit1 = H2ODeepLearningEstimator(model_id='dl_fit1', seed=1)
dl_fit1.train(x=x, y=y, training_frame=train)

deeplearning Model Build progress: |██████████████████████████████████████| 100%


In [36]:
dl_fit2 = H2ODeepLearningEstimator(model_id='dl_fit2', 
                                   epochs=20, 
                                   hidden=[10,10], 
                                   stopping_rounds=0,  #disable early stopping
                                   seed=1)
dl_fit2.train(x=x, y=y, training_frame=train)

deeplearning Model Build progress: |██████████████████████████████████████| 100%



dl_fit3 = H2ODeepLearningEstimator(model_id='dl_fit3', 
                                   epochs=20, 
                                   hidden=[20,20],
                                   score_interval=1,          #used for early stopping
                                   stopping_rounds=3,         #used for early stopping
                                  
                                   stopping_tolerance=0.0005, #used for early stopping
                                   seed=1)
dl_fit3.train(x=x, y=y, training_frame=train, validation_frame=valid)

In [42]:

dl_perf1 = dl_fit1.model_performance(test)
dl_perf2 = dl_fit2.model_performance(test)
dl_perf3 = dl_fit3.model_performance(test)

In [38]:
dl_perf1 #0.7741


ModelMetricsMultinomial: deeplearning
** Reported on test data. **

MSE: 0.17877210132993498
RMSE: 0.42281449990502334
LogLoss: 0.5683149968245718
Mean Per-Class Error: 0.3629151282734375
Confusion Matrix: vertical: actual; across: predicted



functional,functional needs repair,non functional,Error,Rate
4365.0,168.0,253.0,0.0879649,"421 / 4,786"
360.0,211.0,68.0,0.6697966,428 / 639
1073.0,101.0,2373.0,0.3309839,"1,174 / 3,547"
5798.0,480.0,2694.0,0.2254793,"2,023 / 8,972"


Top-3 Hit Ratios: 


k,hit_ratio
1,0.7745207
2,0.9460544
3,1.0


In [39]:
dl_perf2


ModelMetricsMultinomial: deeplearning
** Reported on test data. **

MSE: 0.1867986498852388
RMSE: 0.43220209380015595
LogLoss: 0.5853762009576046
Mean Per-Class Error: 0.43303638892772794
Confusion Matrix: vertical: actual; across: predicted



functional,functional needs repair,non functional,Error,Rate
4359.0,39.0,388.0,0.0892186,"427 / 4,786"
436.0,79.0,124.0,0.8763693,560 / 639
1164.0,19.0,2364.0,0.3335213,"1,183 / 3,547"
5959.0,137.0,2876.0,0.2418636,"2,170 / 8,972"


Top-3 Hit Ratios: 


k,hit_ratio
1,0.7581364
2,0.9450513
3,1.0


In [43]:
    dl_perf3


ModelMetricsMultinomial: deeplearning
** Reported on test data. **

MSE: 0.17970954372798933
RMSE: 0.423921624510934
LogLoss: 0.5693096598332138
Mean Per-Class Error: 0.39301745899821366
Confusion Matrix: vertical: actual; across: predicted



functional,functional needs repair,non functional,Error,Rate
4282.0,79.0,425.0,0.1053071,"504 / 4,786"
406.0,146.0,87.0,0.7715180,493 / 639
1028.0,44.0,2475.0,0.3022272,"1,072 / 3,547"
5716.0,269.0,2987.0,0.2306063,"2,069 / 8,972"


Top-3 Hit Ratios: 


k,hit_ratio
1,0.7693937
2,0.9502898
3,1.0


dl_fit3.scoring_history()

gbm grid search

In [16]:
from h2o.grid.grid_search import H2OGridSearch

In [17]:
gbm_params1 = {'learn_rate': [0.01, 0.1], 
                'max_depth': [3, 5, 9],
                'sample_rate': [0.8, 1.0],
                'col_sample_rate': [0.2, 0.5, 1.0]}

In [20]:
gbm_grid1 = H2OGridSearch(model=H2OGradientBoostingEstimator,
                          grid_id='gbm_grid1',
                          hyper_params=gbm_params1)
gbm_grid1.train(x=x, y=y, 
                training_frame=train, 
                validation_frame=valid, 
                ntrees=100,
                seed=1)

gbm Grid Build progress: |████████████████████████████████████████████████| 100%


In [22]:
gbm_gridperf = gbm_grid1.get_grid

In [23]:
print(gbm_gridperf)

     col_sample_rate learn_rate max_depth sample_rate           model_ids  \
0                0.5        0.1         9         1.0  gbm_grid1_model_34   
1                0.2        0.1         9         1.0  gbm_grid1_model_33   
2                1.0        0.1         9         1.0  gbm_grid1_model_35   
3                0.2        0.1         9         0.8  gbm_grid1_model_15   
4                0.5        0.1         9         0.8  gbm_grid1_model_16   
5                1.0        0.1         9         0.8  gbm_grid1_model_17   
6                0.5        0.1         5         1.0  gbm_grid1_model_28   
7                1.0        0.1         5         1.0  gbm_grid1_model_29   
8                0.2        0.1         5         1.0  gbm_grid1_model_27   
9                1.0        0.1         5         0.8  gbm_grid1_model_11   
10               0.5        0.1         5         0.8  gbm_grid1_model_10   
11               0.2        0.1         5         0.8   gbm_grid1_model_9   